In [ ]:
# %pip install datasets
from datasets import load_dataset
imdbData = load_dataset('imdb')
print(type(imdbData))

In [ ]:
imdbData['train'][10]

In [ ]:
#%pip install textblob
from textblob import TextBlob

In [ ]:
senti_score = TextBlob(imdbData['train'][0]['text'])
print(senti_score.sentiment)

In [ ]:
# %pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
print(analyzer.polarity_scores(imdbData['train'][0]['text']))

In [ ]:
# %pip uninstall transformers -y
# %pip install transformers
import torch
import transformers
from transformers import pipeline

In [ ]:
senti_pipeline = pipeline('sentiment-analysis')
result = senti_pipeline(imdbData['train'][0]['text'])
print(result)

In [ ]:
BertBaseModel = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')
result2 = BertBaseModel(imdbData['train'][0]['text'])
print(result2)

In [ ]:
DistilBertModel = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

result = DistilBertModel(imdbData["train"][0]["text"])
print(result)

In [ ]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

inputs = tokenizer(
    imdbData["train"][0]["text"],
    return_tensors="pt",
    truncation=True,
    max_length=512
)

outputs = model(**inputs)
last_hidden_state = outputs.last_hidden_state
print(last_hidden_state.shape)


In [ ]:
import pandas as pd

type(imdbData['train'][0])

rows = []

for token in imdbData['train']:
    rows.append({
            'Text': token['text'],
            'Label': token['label']
        })

imdbDF = pd.DataFrame(rows, columns=['Text', 'Label'])
print(imdbDF.head())
print("-" * 20)
print(imdbDF.info())


In [ ]:
import re

imdbDF['CleanedUpReviewText'] = imdbDF['Text'].str.lower()
imdbDF['CleanedUpReviewText'] = imdbDF.apply(
    lambda x: re.sub(r"([^A-Za-z\s])", "",
                     x['CleanedUpReviewText']
    ), axis=1
)
print(imdbDF.head())

In [ ]:
vaderSentimentAnalyzer = SentimentIntensityAnalyzer()
imdbDF['vaderSentimentScore'] = imdbDF['CleanedUpReviewText'].apply(
    lambda x: vaderSentimentAnalyzer.polarity_scores(x)['compound']
)

print(imdbDF.head())

bins = [-1, -0.1, 0.1, 1]
labels = ['negative', 'neutral', 'positive']
imdbDF['vaderSentimentLabel'] = pd.cut(
    imdbDF['vaderSentimentScore'],
    bins=bins,
    labels=labels
)
imdbDF['vaderSentimentLabel'].value_counts().plot(kind='bar')

In [ ]:
transformer_labels = []

for review in imdbDF['CleanedUpReviewText'].values:
    # print(review)
    result = senti_pipeline(review, truncation=True, max_length=512)
    transformer_labels.append(result[0]['label'])

imdbDF['transformerLabels'] = transformer_labels

print(imdbDF['transformerLabels'])

In [ ]:
imdbDF['transformerLabels'].value_counts().plot(kind='bar')

### __Bag Of Words__

In [ ]:
reviewArray = imdbDF['Text'].values


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
countVec = CountVectorizer()

countVecFit = countVec.fit_transform(reviewArray)
bagOfWords = pd.DataFrame(
    countVecFit.toarray(),
    columns=countVec.get_feature_names_out()
)

In [ ]:
print(bagOfWords.head())

In [ ]:
print(imdbDF.columns)

In [ ]:

imdbDF['CleannedReview'] = imdbDF.apply(
    lambda x: re.sub(r"([^A-Za-z\s])", "",
                     x['Text']
    ), axis=1
)



In [ ]:
print(type(imdbDF['CleannedReview']))
reviewArray = imdbDF['CleannedReview'].values[0:1000]
print(type(reviewArray))
print(reviewArray.shape)


countVecFit = countVec.fit_transform(reviewArray)
bagOfWords = pd.DataFrame(
    countVecFit.toarray(),
    columns=countVec.get_feature_names_out()
)

In [ ]:
bagOfWords.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVec = TfidfVectorizer()
tfidfFit = tfidfVec.fit_transform(reviewArray)

In [ ]:
tfidf_bag = pd.DataFrame(
    tfidfFit.toarray(),
    columns=tfidfVec.get_feature_names_out()
)

In [ ]:
print(tfidf_bag)